Import the model that we will use in order to do image to text conversion

In [5]:
pip install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [1]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] =  userdata.get('OPENAI')

In [2]:
import base64
import requests

# OpenAI API Key
api_key = userdata.get('OPENAI')

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "/content/Botrytis-Blight-on-Roses.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

prompt_template = (
    "Analyze the provided image of the plant and identify the following details:\n\n"
    "1. **Plant Part**: Determine whether the image shows a flower, leaves, stem, or other parts of the plant.\n"
    "2. **Type of Flower or Plant**: If the flower is visible, describe its species or common name. If other parts of the plant are shown, provide identification if possible.\n"
    "3. **Color and Appearance**: Describe the colors and any noticeable features of the plant part in the image (e.g., spots, stripes, shape of leaves).\n"
    "4. **Condition Assessment**:\n"
    "   - **Health Status**: Indicate if the plant part appears healthy, wilting, or diseased.\n"
    "   - **Stage of Life**: Specify if the plant is in a budding, blooming, or wilting stage, if applicable.\n"
    "   - **Damage or Anomalies**: Mention any signs of damage, pests, or unusual growth patterns.\n"
    "5. **Environmental Context** (if visible in the image): Comment on the surrounding environment, such as garden, wild, indoor, etc.\n\n"
    "Please provide a concise, clear, and accurate description based on the visual information present in the image. "
    "Utilize descriptive language and avoid ambiguous terms. Ensure the response is tailored to the specifics of the image, "
    "reflecting the details of the plant part and its condition accurately."
)


payload = {
  "model": "gpt-4-vision-preview",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": prompt_template
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-8S4q78wUduZGACQixEVKUX76QlFQp', 'object': 'chat.completion', 'created': 1701702479, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 1040, 'completion_tokens': 297, 'total_tokens': 1337}, 'choices': [{'message': {'role': 'assistant', 'content': "1. **Plant Part**: The image shows a flower, specifically a rose.\n\n2. **Type of Flower or Plant**: The flower in the image is a rose. However, without more information, it's not possible to determine the exact species or cultivar.\n\n3. **Color and Appearance**: The rose appears to have originally been a light color, probably a shade of pale pink or peach, but the petals have browned significantly, indicating decay. The flower is fully open, with multiple layers of petals that spiral inward. Some petals have brown edges and spots, and there are signs of wilting and drying, particularly on the outer petals.\n\n4. **Condition Assessment**:\n   - **Health Status**: The flower appears to be wilting and showing sig

In [3]:
json_response = response.json()
result = json_response['choices'][0]['message']['content']
result

"1. **Plant Part**: The image shows a flower, specifically a rose.\n\n2. **Type of Flower or Plant**: The flower in the image is a rose. However, without more information, it's not possible to determine the exact species or cultivar.\n\n3. **Color and Appearance**: The rose appears to have originally been a light color, probably a shade of pale pink or peach, but the petals have browned significantly, indicating decay. The flower is fully open, with multiple layers of petals that spiral inward. Some petals have brown edges and spots, and there are signs of wilting and drying, particularly on the outer petals.\n\n4. **Condition Assessment**:\n   - **Health Status**: The flower appears to be wilting and showing signs of decay.\n   - **Stage of Life**: Based on the browning and wilting petals, the rose is past its peak blooming stage and is in the process of dying.\n   - **Damage or Anomalies**: There is noticeable browning and drying, especially on the outer petals, which suggests either

In [6]:
from langchain.chains import create_extraction_chain
from langchain.chat_models import ChatOpenAI

# Schema
schema = {
    "properties": {
        "plant": {"type": "string"},
         "part_of_plant": {"type": "string"},
        "condition": {"type": "string"},
        "explanation_of_condition":{"type":"string"},
    },
    "required": ["plant"],
}
# Run chain
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
chain = create_extraction_chain(schema, llm)
structured_data = chain.run(result)[0]
structured_data

{'plant': 'rose',
 'part_of_plant': 'flower',
 'condition': 'wilting and showing signs of decay',
 'explanation_of_condition': 'The flower appears to be wilting and showing signs of decay.'}

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser


prompt = ChatPromptTemplate.from_template((
    "Given the following details about a plant, provide recommendations for its care:\n\n"
    "Plant Type: {plant}\n"
    "Visible Plant Parts: {part_of_plant}\n"
    "Condition Assessment:\n"
    "   - Overall Health: {condition}\n"
    "   - Specific Issues: {explanation_of_condition}\n\n"
    "Based on the above information, please provide a comprehensive care guide for this plant. The guide should include:\n"
    "   - Watering recommendations\n"
    "   - Sunlight requirements\n"
    "   - Soil and nutrient needs\n"
    "   - Pest control suggestions\n"
    "   - Any other relevant care tips specific to the plant type and its current condition."))
model = ChatOpenAI(temperature=0.2, model="gpt-3.5-turbo", max_tokens=800)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

recomendation = chain.invoke(structured_data)
print(recomendation)

Care Guide for a Wilting and Decaying Rose Plant:

Watering Recommendations:
- Ensure the rose plant is receiving adequate water. Water deeply and thoroughly, allowing the soil to dry slightly between waterings.
- Avoid overwatering, as this can lead to root rot and further decay. Check the moisture level of the soil before watering.

Sunlight Requirements:
- Roses generally require at least 6 hours of direct sunlight per day. Ensure the plant is placed in a location that receives ample sunlight.

Soil and Nutrient Needs:
- Roses prefer well-draining soil with a pH level between 6.0 and 6.5. Amend the soil with organic matter, such as compost or well-rotted manure, to improve drainage and fertility.
- Apply a balanced rose fertilizer according to the package instructions, typically every 4-6 weeks during the growing season.

Pest Control Suggestions:
- Inspect the plant regularly for common rose pests such as aphids, spider mites, and rose slugs. Remove any visible pests by hand or use